In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline

/home/ptyagi/miniconda3/envs/thesis_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
tweet_replies_feb_2019_en = pd.read_csv('/work/ptyagi/masterthesis/tmp/tweet_replies_feb_2019_en.csv')

In [13]:
tweet_replies_feb_2019_en.drop(columns=['Unnamed: 0'], inplace=True)

tweet_replies_feb_2019_en['replies'] = tweet_replies_feb_2019_en['replies'].astype(str).fillna('')

tweet_replies_feb_2019_en['replies'] = tweet_replies_feb_2019_en['replies'].replace('nan', '')

In [14]:
tweet_replies_feb_2019_en

,id,conversation_id,created_at,replies,tweet_text,tweet_lang,lang_confidence
0,1091717834860826624,1091717129995542528,2019-02-02 15:19:49+00:00,Go Trump!!,Energy Insider: Exploring the impact of winter...,en,0.742598
1,1092776979567468546,1092776506106015746,2019-02-05 13:28:29+00:00,eat local and grow local the ecosystem is buil...,Given how what we eat (demand) determines what...,en,0.901509
2,1092835036037087232,1092784983998189568,2019-02-05 17:19:11+00:00,TUMP,New: [USER] PhD candidate Rodrigo Muñoz-Gonzál...,en,0.924515
3,1093139326496636928,1093132427810271233,2019-02-06 13:28:19+00:00,the sky is falling the sky in falling the corr...,See how Canada is addressing #ClimateChange th...,en,0.738337
4,1094072089672175622,1094070997135822848,2019-02-09 03:14:47+00:00,you are beyond demented the utter presumptuous...,You can kiss your beer goodbye if climate chan...,en,0.934737
...,...,...,...,...,...,...,...
12637,1101270343597998080,1101215748305944576,2019-02-28 23:58:05+00:00,i do not see them moving over is there a repub...,"For an embarrassingly long time, Dems used “sc...",en,0.895219
12638,1101270588180582402,1101265867353018368,2019-02-28 23:59:03+00:00,shriver unaware we are already importing much ...,Time for some climate change chat #bbcqt [URL],en,0.568811
12639,1101270664588288000,1098626225201922049,2019-02-28 23:59:21+00:00,keep laptops amp all wireless fa away from u a...,Join me and climate kitty for a [USER] AMA on ...,en,0.884687
12640,1101270692002217984,1101215748305944576,2019-02-28 23:59:28+00:00,actually it makes you more easy to dismiss kee...,"For an embarrassingly long time, Dems used “sc...",en,0.895219


In [15]:
emotion_classifier = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-emotion-latest", device=2)

In [16]:
emotion_predictions = emotion_classifier(tweet_replies_feb_2019_en['replies'].tolist(), batch_size=8)

In [17]:
emotion_predictions[:5] # type: ignore

[{'label': 'anger', 'score': 0.9626848101615906},
 {'label': 'optimism', 'score': 0.9835257530212402},
 {'label': 'anger', 'score': 0.3523235321044922},
 {'label': 'optimism', 'score': 0.8389987945556641},
 {'label': 'anger', 'score': 0.9850420355796814}]

In [18]:
tweet_replies_feb_2019_en['reply_emotion_roberta_base'] = [pred['label'] for pred in emotion_predictions] # type: ignore
tweet_replies_feb_2019_en['emotion_confidence_roberta_base'] = [pred['score'] for pred in emotion_predictions] # type: ignore

In [19]:
tweet_replies_feb_2019_en.head()

,id,conversation_id,created_at,replies,tweet_text,tweet_lang,lang_confidence,reply_emotion_roberta_base,emotion_confidence_roberta_base
0,1091717834860826624,1091717129995542528,2019-02-02 15:19:49+00:00,Go Trump!!,Energy Insider: Exploring the impact of winter...,en,0.742598,anger,0.962685
1,1092776979567468546,1092776506106015746,2019-02-05 13:28:29+00:00,eat local and grow local the ecosystem is buil...,Given how what we eat (demand) determines what...,en,0.901509,optimism,0.983526
2,1092835036037087232,1092784983998189568,2019-02-05 17:19:11+00:00,TUMP,New: [USER] PhD candidate Rodrigo Muñoz-Gonzál...,en,0.924515,anger,0.352324
3,1093139326496636928,1093132427810271233,2019-02-06 13:28:19+00:00,the sky is falling the sky in falling the corr...,See how Canada is addressing #ClimateChange th...,en,0.738337,optimism,0.838999
4,1094072089672175622,1094070997135822848,2019-02-09 03:14:47+00:00,you are beyond demented the utter presumptuous...,You can kiss your beer goodbye if climate chan...,en,0.934737,anger,0.985042


In [21]:
print(tweet_replies_feb_2019_en.query("conversation_id == 1092776506106015746")['replies'].to_list())
print(tweet_replies_feb_2019_en.query("conversation_id == 1092776506106015746")['reply_emotion_roberta_base'].to_list())

['eat local and grow local the ecosystem is built like that and would continue to thrive']
['optimism']


In [22]:
emotion_classifier_l = pipeline("text-classification", model="cardiffnlp/twitter-roberta-large-emotion-latest", device=2)

In [23]:
emotion_predictions = emotion_classifier_l(tweet_replies_feb_2019_en['replies'].tolist(), batch_size=8)

In [24]:
emotion_predictions[:5] # type: ignore

[{'label': 'anger', 'score': 0.9726458191871643},
 {'label': 'optimism', 'score': 0.9806329607963562},
 {'label': 'joy', 'score': 0.700554370880127},
 {'label': 'joy', 'score': 0.9542987942695618},
 {'label': 'anger', 'score': 0.9939709901809692}]

In [25]:
tweet_replies_feb_2019_en['reply_emotion_roberta_large'] = [pred['label'] for pred in emotion_predictions] # type: ignore
tweet_replies_feb_2019_en['emotion_confidence_roberta_large'] = [pred['score'] for pred in emotion_predictions] # type: ignore

In [26]:
tweet_replies_feb_2019_en.head()

,id,conversation_id,created_at,replies,tweet_text,tweet_lang,lang_confidence,reply_emotion_roberta_base,emotion_confidence_roberta_base,reply_emotion_roberta_large,emotion_confidence_roberta_large
0,1091717834860826624,1091717129995542528,2019-02-02 15:19:49+00:00,Go Trump!!,Energy Insider: Exploring the impact of winter...,en,0.742598,anger,0.962685,anger,0.972646
1,1092776979567468546,1092776506106015746,2019-02-05 13:28:29+00:00,eat local and grow local the ecosystem is buil...,Given how what we eat (demand) determines what...,en,0.901509,optimism,0.983526,optimism,0.980633
2,1092835036037087232,1092784983998189568,2019-02-05 17:19:11+00:00,TUMP,New: [USER] PhD candidate Rodrigo Muñoz-Gonzál...,en,0.924515,anger,0.352324,joy,0.700554
3,1093139326496636928,1093132427810271233,2019-02-06 13:28:19+00:00,the sky is falling the sky in falling the corr...,See how Canada is addressing #ClimateChange th...,en,0.738337,optimism,0.838999,joy,0.954299
4,1094072089672175622,1094070997135822848,2019-02-09 03:14:47+00:00,you are beyond demented the utter presumptuous...,You can kiss your beer goodbye if climate chan...,en,0.934737,anger,0.985042,anger,0.993971


In [28]:
print(tweet_replies_feb_2019_en.query("conversation_id == 1093132427810271233")['replies'].to_list())
print(tweet_replies_feb_2019_en.query("conversation_id == 1093132427810271233")['reply_emotion_roberta_large'].to_list())

['the sky is falling the sky in falling the corrupt un says the world will end in so lets party like its']
['joy']


In [29]:
tweet_replies_feb_2019_en.to_csv('/work/ptyagi/masterthesis/tmp/tweet_replies_predictions_feb_2019_en.csv')